In [50]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#from statsmodels import robust
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import os
import matplotlib.cm as cm

# run pca and visualize

import scipy

colors = [
'black','blue','red','green','cyan','magenta','brown','pink',
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']

sorted_colors=colors

#import deconv

# visualize recomputed templates over time;
def binary_reader_waveforms(filename, n_channels, n_times, spikes, channels=None, data_type='float32'):
    ''' Reader for loading raw binaries
    
        standardized_filename:  name of file contianing the raw binary
        n_channels:  number of channels in the raw binary recording 
        n_times:  length of waveform 
        spikes: 1D array containing spike times in sample rate of raw data
        channels: load specific channels only
        data_type: float32 for standardized data
    
    '''

    # ***** LOAD RAW RECORDING *****
    if channels is None:
        wfs = np.zeros((spikes.shape[0], n_times, n_channels), data_type)
    else:
        wfs = np.zeros((spikes.shape[0], n_times, channels.shape[0]), data_type)
    if data_type =='float32':
        data_len = 4
    else:
        data_len = 2

    with open(filename, "rb") as fin:
        for ctr,s in enumerate(spikes):
            #print (ctr,s)
            # index into binary file: time steps * 4  4byte floats * n_channels
            fin.seek(s * data_len * n_channels, os.SEEK_SET)
            wfs[ctr] = np.fromfile(
                fin,
                dtype=data_type,
                count=(n_times * n_channels)).reshape(n_times, n_channels)[:,channels]
    fin.close()
    return wfs

Autosaving every 180 seconds


In [6]:
times = np.load('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/spike_times.npy')
print (times)
ids = np.load('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/spike_clusters.npy')
print (ids)

spike_train = np.hstack((times,ids))
print (spike_train.shape)

[[      86]
 [     116]
 [     140]
 ...
 [63091083]
 [63091086]
 [63091105]]
[[139]
 [ 78]
 [ 68]
 ...
 [184]
 [154]
 [126]]
(1531845, 2)


In [7]:
# load raw data and spike train
n_chan=385
data = np.fromfile('/media/cat/2TB/nick_steinmetz_drift/rec1/tmp/preprocess/standardized.bin','float32', count=n_chan*300*30000)
print (data.shape)

#n_chan = 512
data2D = data.reshape(-1,n_chan)
print (data2D.shape)

#spike_train = np.load('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/tmp/output/spike_train.npy')
#print (spike_train)

geom = np.loadtxt('/media/cat/2TB/nick_steinmetz_drift/rec1/p1_g0_t0.imec0.ap_geom.txt')
print (geom.shape)


(3465000000,)
(9000000, 385)
(385, 2)


In [25]:
spike_clusters = np.load('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/spike_clusters.npy')
spike_times = np.load('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/spike_times.npy')

spike_train = np.zeros((spike_clusters.shape[0],2),'int32')
spike_train[:,0] = spike_times.squeeze()
ctr = 0
for unit in np.unique(spike_clusters):
    if ctr%50==0:
        print (ctr)
    idx = np.where(spike_clusters==unit)[0]
    spike_train[idx,1] = ctr
    ctr+=1
    
print (spike_train)
np.save('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/spike_train_reloaded.npy',spike_train)

0
50
100
150
[[      86      134]
 [     116       76]
 [     140       67]
 ...
 [63091083      179]
 [63091086      149]
 [63091105      122]]


In [9]:
# use int16 original binary
if False:
    if False:
        filename = '/media/cat/12TB/dbox/Dropbox/data_temp/liam/julien/neuropixels/nick_steinmetz/p1_g0_t0.imec0.ap.bin'
        data_type = 'int16'
    # use float32 standardized file
    else:
        filename = '/media/cat/14TB/liam/512chan/2005-04-26-0/data002/tmp/preprocess/standardized.bin'
        data_type = 'float32'

    n_channels = 512
    n_times = 101
    # spike_times = np.load('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/spike_times.npy')
    # spike_clusters = np.load('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/spike_clusters.npy')
    #units = np.unique(spike_clusters)
    spike_train = np.load('/media/cat/14TB/liam/512chan/2005-04-26-0/data002/kilosort2/spike_times_ordered.npy')

    channels = np.arange(n_channels)

    #for unit in np.unique(spike_clusters):
    #for unit in np.unique(spike_clusters)[:]:
    #units = [0]
    units = np.unique(spike_train[:,1])
    print ("units; ", units.shape)
    templates = []
    for unit in units:
        if unit%10==0:
            print (unit)
        idx = np.where(spike_train[:,1]==unit)[0]
        spikes = np.int32(spike_train[idx,0])-n_times//2
        # sub sample spikes to speed up loading
        idx = np.where(spikes<5*60*20000-100)[0]
        spikes = spikes[idx]

        idx = np.random.choice(np.arange(spikes.shape[0]),min(1000,spikes.shape[0]))
        #print (idx)
        spikes=spikes[idx]

        #spikes = spikes[::10]
        wfs = binary_reader_waveforms(filename, n_channels, n_times, spikes, channels, data_type)
        print (unit, wfs.shape)
        if wfs.shape[0]==0:
            templates.append(np.zeros((n_times,n_channels),'float32'))
        else:
            templates.append(wfs.mean(0))

    
# max_chan = wfs.mean(0).ptp(0).argmax(0)
# fig = plt.figure()
# plt.plot(wfs[:,:,max_chan].T, c='black',alpha=.02)
# plt.plot(wfs.mean(0)[:,max_chan],c='red')
# plt.show()


In [26]:
# load raw data and spike train
n_chan = 385
data = np.fromfile('/media/cat/2TB/nick_steinmetz_drift/rec1/tmp/preprocess/standardized.bin','float32',count=300*n_chan*30000)
print (data.shape)

n_chan = 385
data2D = data.reshape(-1,n_chan)
print (data2D.shape)

spike_train = np.load('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/spike_train_reloaded.npy')
print (spike_train)

geom = np.loadtxt('/media/cat/2TB/nick_steinmetz_drift/rec1/p1_g0_t0.imec0.ap_geom.txt')
print (geom.shape)

temps = np.load('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/templates_reloaded.npy')

(3465000000,)
(9000000, 385)
[[      86      134]
 [     116       76]
 [     140       67]
 ...
 [63091083      179]
 [63091086      149]
 [63091105      122]]
(385, 2)


In [38]:
temps = np.load('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/templates_reloaded.npy')
print (temps.shape)
print (np.unique(spike_train[:,1]))

# st2 = np.zeros((0,2), 'int32')
# ctr=0
# for k in range(temps.shape[0]):
#     idx = np.where(spike_train[:,1]==k)[0]
#     times = spike_train[idx,0]
#     ids = times*0+ctr
    
#     sp_local = np.vstack((times, ids)).T
#     print (sp_local.shape)
#     st2 = np.vstack((st2,sp_local))
        
#     ctr+=1
# st2 = np.int32(st2)
# print (st2)

# spike_train = st2.copy()
np.save('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/spike_train.npy', spike_train)

(191, 101, 385)
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190]


In [34]:

def binary_reader_waveforms(filename, n_channels, n_times, spikes, channels=None, data_type='float32'):
    ''' Reader for loading raw binaries
    
        standardized_filename:  name of file contianing the raw binary
        n_channels:  number of channels in the raw binary recording 
        n_times:  length of waveform 
        spikes: 1D array containing spike times in sample rate of raw data
        channels: load specific channels only
        data_type: float32 for standardized data
    
    '''

    # ***** LOAD RAW RECORDING *****
    if channels is None:
        wfs = np.zeros((spikes.shape[0], n_times, n_channels), data_type)
    else:
        wfs = np.zeros((spikes.shape[0], n_times, channels.shape[0]), data_type)
    if data_type =='float32':
        data_len = 4
    else:
        data_len = 2
    with open(filename, "rb") as fin:
        for ctr,s in enumerate(spikes):
            # index into binary file: time steps * 4  4byte floats * n_channels
            fin.seek(s * data_len * n_channels, os.SEEK_SET)
            wfs[ctr] = np.fromfile(
                fin,
                dtype='float32',
                count=(n_times * n_channels)).reshape(n_times, n_channels)[:,channels]

    fin.close()
    return wfs

#  load templates from spike trains
import numpy as np
import os
n_channels = 385
n_times = 101
channels = np.arange(n_channels)

#fname_standardized = '/media/cat/4TBSSD/liam/512channels/2009-04-13-5_120mins/tmp/final_deconv/residual/residual.bin'
fname_standardized = '/media/cat/2TB/nick_steinmetz_drift/rec1/tmp/preprocess/standardized.bin'

spike_train = np.load('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/spike_train_reloaded.npy')

units = np.unique(spike_train[:,1])

# 
templates = []
for unit in units:
    idx = np.where(spike_train[:,1]==unit)[0]
    spikes = spike_train[idx,0]-30
    idx = np.where(spikes<(300*30000-100))[0]
    spikes=spikes[idx]
    wfs = binary_reader_waveforms(fname_standardized, n_channels, n_times, spikes, channels, data_type ='float32')
    print ("loading unit: ", unit,  wfs.shape)
    if wfs.shape[0]==0:
        temp = np.zeros((n_times,n_channels))
    else:
        temp = wfs.mean(0)
    
    templates.append(temp)

templates = np.array(templates)
print (templates.shape)

loading unit:  0 (193, 101, 385)
loading unit:  1 (444, 101, 385)
loading unit:  2 (546, 101, 385)
loading unit:  3 (689, 101, 385)
loading unit:  4 (1044, 101, 385)
loading unit:  5 (148, 101, 385)
loading unit:  6 (132, 101, 385)
loading unit:  7 (645, 101, 385)
loading unit:  8 (1548, 101, 385)
loading unit:  9 (6, 101, 385)
loading unit:  10 (544, 101, 385)
loading unit:  11 (163, 101, 385)
loading unit:  12 (638, 101, 385)
loading unit:  13 (4, 101, 385)
loading unit:  14 (921, 101, 385)
loading unit:  15 (864, 101, 385)
loading unit:  16 (3072, 101, 385)
loading unit:  17 (1591, 101, 385)
loading unit:  18 (565, 101, 385)
loading unit:  19 (740, 101, 385)
loading unit:  20 (383, 101, 385)
loading unit:  21 (1424, 101, 385)
loading unit:  22 (4320, 101, 385)
loading unit:  23 (331, 101, 385)
loading unit:  24 (148, 101, 385)
loading unit:  25 (5732, 101, 385)
loading unit:  26 (632, 101, 385)
loading unit:  27 (0, 101, 385)
loading unit:  28 (264, 101, 385)
loading unit:  29 (48, 

/home/cat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Mean of empty slice.
/home/cat/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


loading unit:  31 (731, 101, 385)
loading unit:  32 (420, 101, 385)
loading unit:  33 (893, 101, 385)
loading unit:  34 (308, 101, 385)
loading unit:  35 (7, 101, 385)
loading unit:  36 (113, 101, 385)
loading unit:  37 (942, 101, 385)
loading unit:  38 (314, 101, 385)
loading unit:  39 (358, 101, 385)
loading unit:  40 (165, 101, 385)
loading unit:  41 (0, 101, 385)
loading unit:  42 (493, 101, 385)
loading unit:  43 (12, 101, 385)
loading unit:  44 (480, 101, 385)
loading unit:  45 (1527, 101, 385)
loading unit:  46 (49, 101, 385)
loading unit:  47 (433, 101, 385)
loading unit:  48 (842, 101, 385)
loading unit:  49 (2346, 101, 385)
loading unit:  50 (1179, 101, 385)
loading unit:  51 (63, 101, 385)
loading unit:  52 (57, 101, 385)
loading unit:  53 (1996, 101, 385)
loading unit:  54 (200, 101, 385)
loading unit:  55 (720, 101, 385)
loading unit:  56 (504, 101, 385)
loading unit:  57 (630, 101, 385)
loading unit:  58 (322, 101, 385)
loading unit:  59 (57, 101, 385)
loading unit:  60 (

In [55]:
#np.save('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/templates_reloaded.npy',templates)
temps = np.load('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/templates_reloaded.npy')

In [56]:
# save channels with > 5 templates on them that are > 8.0 SU
#temps = np.load('/media/cat/14TB/liam/512chan/2005-04-26-0/data002/kilosort2/templates_reloaded.npy')
print (temps.shape)

ptps = temps.ptp(1).max(1)
max_chans = temps.ptp(1).argmax(1)


#root_dir = '/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/ari/julien_real_neighbour_chans/'
root_dir = '/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/ari/julien_real_neighbour_chans_neuropixels2'

ctr=0
for chan in range(n_chan):
    idx2 = np.where(max_chans==chan)[0]
    ptps_local = ptps[idx2]

    locs = geom[chan]
    
    thresh=70
    chans_neighbour = []
    for k in range(n_chan):
        dist = np.linalg.norm(locs-geom[k])
        if dist < thresh:
            chans_neighbour.append(k)

            
    print (chans_neighbour)
    #break
    # all chans within neighbourhood
    chans_neighbour = np.unique(chans_neighbour)
       
    #idx3 = np.where(ptps_local>8.0)[0]
    
    
    if idx2.shape[0]>3:
        print (chan)
                
        # save this channel and the data
        np.save(root_dir+'/chan_'+str(ctr)+'_data.npy',
                data2D[:,chan])
        
        # save templates
        temps_out=[]
        for ch in chans_neighbour:
            idx = np.where(max_chans==ch)[0]
            temps_out.extend(temps[idx,:,chan])  #Make sure you save max channel tempalte NOT 
            
        temps_out = np.array(temps_out)[:,20:95]
        print (temps_out.shape)
            
        np.save(root_dir+'/chan_'+str(ctr)+'_templates.npy',
                temps_out)
        
        # save spiketrains
        times_out= []
        for ch in chans_neighbour:
            #idx = np.where
            ids = np.where(max_chans==ch)[0]
            for id_ in ids:
                idx4 = np.where(spike_train[:,1]==id_)[0]
                times = spike_train[idx4,0]
                idx5 = np.where(times<(5*60*20000))[0]
                times_out.append(times[idx5])
        
        np.save(root_dir+'/chan_'+str(ctr)+'_times.npy',
                times_out)
        #break    
        ctr+=1

(191, 101, 385)
[0, 1, 2, 3, 4, 5, 6]
0
(7, 75)
[0, 1, 2, 3, 4, 5, 6, 7]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13]
[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17]
[6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21]
[10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25]
[14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
[

[142, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154]
[142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]
[144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157]
[145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157]
[146, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158]
[146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159]
[148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161]
154
(17, 75)
[149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 161]
[150, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162]
[150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163]
[152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165]
[153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 165]
[154, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166]
[154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167]
[156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 16

[281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294]
[283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296]
[284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 296]
[285, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297]
[285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298]
[287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300]
[288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300]
[289, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301]
[289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302]
[291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304]
[292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 304]
[293, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305]
[293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306]
[295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308]
[296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 308]
[297,

In [64]:
data = np.load('/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/ari/julien_real_neighbour_chans_neuropixels2/chan_5_data.npy')
templates = np.load('/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/ari/julien_real_neighbour_chans_neuropixels2/chan_5_templates.npy',allow_pickle=True)
times = np.load('/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/ari/julien_real_neighbour_chans_neuropixels2/chan_5_times.npy',allow_pickle=True)
plt.plot(data[:10000])
for p in range(templates.shape[0]):
    idx = np.where(times[p]<10000)[0]
    print (idx.shape)
    for k in range(idx.shape[0]):
        plt.plot(times[p][k]+np.arange(75)-9,templates[p],c=colors[p])
plt.show()

(0,)
(0,)
(3,)
(0,)
(0,)
(0,)
(0,)
(0,)
(1,)
(2,)
(1,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(2,)
(3,)
(0,)
(0,)
(0,)
(2,)


In [8]:
# YASS DATASETS
# load raw data and spike train
data = np.fromfile('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/tmp/preprocess/standardized.bin','float32')
print (data.shape)

n_chan = 512
data2D = data.reshape(-1,n_chan)
print (data2D.shape)

spike_train = np.load('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/tmp/output/spike_train.npy')
print (spike_train)

(3072000000,)
(6000000, 512)
[[        0      1430]
 [        0       490]
 [        0       565]
 ...
 [143999995       239]
 [143999996       812]
 [143999997        17]]


In [26]:
# save channels with > 5 templates on them that are > 8.0 SU
out_dir = '/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/ari/ari_real/2009_yass/'

temps = np.load('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/tmp/output/templates/templates_0sec.npy')
print (temps.shape)

ptps = temps.ptp(1).max(1)
max_chans = temps.ptp(1).argmax(1)

ctr=0
for chan in range(512):
    idx2 = np.where(max_chans==chan)[0]
    ptps_local = ptps[idx2]
    idx3 = np.where(ptps_local>8.0)[0]
    #
    
    if idx3.shape[0]>5:
        print (chan, idx2.shape, idx3.shape)
                
        # save this channel and the data
        np.save(out_dir+'/chan_'+str(ctr)+'_data.npy',
                data2D[:,chan])
        
        # save templates
        np.save(out_dir+'/chan_'+str(ctr)+'_templates.npy',
                temps[idx2[idx3],:,chan])
        
        # save spiketrains
        times_out= []
        for id_ in idx2[idx3]:
            idx4 = np.where(spike_train[:,1]==id_)[0]
            times = spike_train[idx4,0]
            idx5 = np.where(times<(5*60*20000))[0]
            times_out.append(times[idx5])
            
        np.save(out_dir+'/chan_'+str(ctr)+'_times.npy',
                times_out)
        ctr+=1     

(2413, 101, 512)
1 (19,) (17,)
8 (11,) (11,)
29 (8,) (7,)
32 (8,) (7,)
49 (12,) (7,)
53 (25,) (24,)
57 (6,) (6,)
60 (7,) (7,)
64 (24,) (22,)
79 (6,) (6,)
80 (7,) (6,)
94 (10,) (7,)
100 (18,) (6,)
108 (7,) (7,)
117 (6,) (6,)
122 (7,) (7,)
123 (9,) (6,)
128 (8,) (7,)
132 (8,) (6,)
133 (9,) (7,)
142 (8,) (8,)
150 (17,) (16,)
158 (13,) (13,)
169 (9,) (9,)
173 (15,) (14,)
186 (27,) (23,)
188 (33,) (29,)
189 (21,) (15,)
190 (32,) (28,)
191 (9,) (7,)
196 (40,) (29,)
199 (17,) (17,)
201 (20,) (12,)
203 (9,) (8,)
214 (7,) (7,)
215 (35,) (33,)
224 (10,) (6,)
225 (8,) (7,)
226 (11,) (10,)
233 (9,) (9,)
241 (18,) (15,)
246 (6,) (6,)
248 (19,) (13,)
253 (8,) (6,)
270 (13,) (13,)
274 (26,) (25,)
275 (11,) (11,)
278 (8,) (7,)
287 (15,) (8,)
289 (6,) (6,)
290 (18,) (9,)
295 (28,) (13,)
298 (10,) (9,)
300 (10,) (9,)
308 (10,) (8,)
315 (13,) (9,)
317 (9,) (7,)
318 (18,) (17,)
333 (11,) (10,)
335 (27,) (22,)
343 (9,) (7,)
350 (12,) (6,)
353 (8,) (6,)
356 (11,) (11,)
367 (7,) (7,)
369 (15,) (13,)
376 (6,)

In [20]:
data = np.fromfile('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/2009-04-13-5.bin','int16',count=512*20000*5*60)
print (data.shape)

(3072000000,)


In [22]:
data.tofile('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/2009-04-13-5_5min.bin')